In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
acq_data = ['Id', 'chan','Name', 'IntRate', 'UnpaidPrinciple', 'LoanDur', 'date_1', 'Payment_one', 'LTV', 'CLTV',
           'Borrow', 'DTI', 'credit_score', 'homeloan','loanreason', 'prop_type', 'No_units', 'status', 'state','address',
           'Mortgage','type','creditscore1', 'Mortgagetype','Mort'];


perf_data = ['Id','month','col3','curintrate','col5','Age','maturity_date','remaining_time','col9','col10','col11','flag','col13',
            'date','lastpaymentdate','closureDate','col17','col18','col19','col20','col21','col22','col23','col24','col25',
            'col26','col27','col28','col29']



In [3]:
processed_acq = pd.read_csv('2000Q1'+'/Acquisition_2000Q1.txt', sep = '|',names = acq_data,index_col=False)
processed_per = pd.read_csv('2000Q1'+'/Performance_2000Q1.txt',sep = '|', names = perf_data, usecols=[0, 15], index_col=False)


IOError: File 2000Q1/Acquisition_2000Q1.txt does not exist

In [4]:
processed_per.drop_duplicates(subset='Id', keep='last', inplace=True)
new_data = pd.merge(processed_acq, processed_per, on='Id', how='inner')

#new_data.rename(index=str, columns={"closureDate": 'Default'}, inplace=True)


In [5]:
new_data['closureDate'].fillna(0,inplace=True)
new_data.loc[new_data['closureDate'] != 0, 'closureDate'] = 1
# length =  len(new_data['closureDate'])
# for i in range(0,length):
#     if new_data['closureDate'][i] != 0:
#         new_data['closureDate'][i] = 1

# new_data['closureDate'] = new_data['closureDate'].astype(int)

In [6]:
new_data['dateMonth'] = new_data['date_1'].apply(lambda a: a.split('/')[0].strip()).astype(int)
new_data['dateYear'] = new_data['date_1'].apply(lambda a: a.split('/')[1].strip()).astype(int)

new_data['paymentMonth'] =  new_data['Payment_one'].apply(lambda a: a.split('/')[0].strip()).astype(int)
new_data['paymentYear'] =  new_data['Payment_one'].apply(lambda a: a.split('/')[1].strip()).astype(int)


new_data.drop(['date_1','Payment_one'], axis=1, inplace =True)


In [7]:
print new_data

In [8]:
new_data.drop(['Mortgage','Mortgagetype','creditscore1','type','Id'], axis=1, inplace=True)
# new_data.drop(['CLTV','LTV'],axis=1,inplace=True)


In [9]:
def getdummies(data):
    col = data.columns[data.isnull().any()]
    new_col = data[col]
    
    
    data.drop(new_col.columns, axis=1, inplace=True)
    objecttype = data.select_dtypes(include=['object'])
    cols = data.drop(objecttype.columns,axis=1)
    
    newdata = pd.DataFrame()
    for i in objecttype.columns:
        temp = pd.get_dummies(objecttype[i],drop_first=True)
        newdata = pd.concat([data,temp],axis=1)
        
    data = pd.concat([cols, newdata, new_col],axis=1).reset_index(drop=True)
    return data
    

In [10]:
from sklearn.metrics import mean_squared_error, r2_score
import sklearn as sk
import matplotlib.pyplot as plt

In [11]:

def random_forest_classifier(features,classification):
    features_train, features_test, classification_train, classification_test = train_test_split(features,classification,test_size=0.25,random_state=0)
    classifier = RandomForestClassifier(n_estimators=200)

    classifier = classifier.fit(features_train,classification_train)
    class_prediction = classifier.predict(features_test)

    x = list(range(len(class_prediction)))

    print class_prediction
    plt.scatter(x,class_prediction,color='red')

    plt.plot(x,classification_test,color='blue')


    plt.xticks(())
    plt.yticks(())
    plt.show()


    print("error mse")
    print(mean_squared_error(classification_test,class_prediction))
    print("variance score: 1 is perfect prediction")
    print(r2_score(classification_test,class_prediction))

    print(classification_report(classification_test,class_prediction))

# features = [[1,3,4,5],[2,67,1,4],[3423,8675,3123,554],[33,67,885,23],[313,367,88512,1223]]

# # classification = [[1],[0],[1],[0]]
# classification = [ 1,0,1,1,1]

# random_forest_classifier(features,classification)

In [12]:
new_val = getdummies(new_data)

In [13]:
print new_val

        IntRate  UnpaidPrinciple  LoanDur  No_units  address dateMonth  \
0         8.000            75000      360         1      173        12   
1         7.875            55000      180         1      208         1   
2         7.750           123000      360         1      630        11   
3         8.500            51000      360         1      316         2   
4         8.750           242000      360         1      335         2   
5         8.250           240000      360         1      483        12   
6         7.500            52000      180         1      347        12   
7         7.625           225000      360         1      980         7   
8         8.000           120000      360         1       63         2   
9         8.000           130000      360         1      300         1   
10        8.375           107000      360         1      633         2   
11        8.000            37000      180         1      286         2   
12        7.875            60000      

In [14]:
# y = new_val['closureDate'].values
# x = new_val.drop(['closureDate'],axis = 1).values


# # features = [[1,3,4,5],[2,67,1,4],[3423,8675,3123,554],[33,67,885,23],[313,367,88512,1223]]

# # # classification = [[1],[0],[1],[0]]
# # classification = [ 1,0,1,1,1]

# # x = [ 1,2]

def run_classfier(data):
    columns = data.columns[data.isnull().any()]
    for each in columns:
        y = data.loc[data[each].notnull(), each].values
        x = data.loc[data[each].notnull()].drop(columns,axis=1).values

        x_test = data.loc[data[each].isnull()].drop(columns,axis=1).values
        if data[each].dtypes == 'object':
            print each 
# random_forest_classifier(x,y)

In [24]:
# y = new_val['closureDate'].values
# x = new_val.drop(['closureDate'],axis = 1).values

# print len(x)
# print x
print new_val['address']
y = new_val['closureDate'].values
# new_val1 = new_val.drop(['Y','CLTV','TLV'],axis=1, inplace=True)
X = new_val['address'].values
# X = new_val.loc[:, new_val.columns != 'closureDate']

random_forest_classifier(X,y)

        address  address
0           173      173
1           208      208
2           630      630
3           316      316
4           335      335
5           483      483
6           347      347
7           980      980
8            63       63
9           300      300
10          633      633
11          286      286
12          346      346
13          334      334
14          491      491
15           70       70
16          977      977
17          450      450
18          917      917
19          871      871
20          451      451
21          330      330
22          812      812
23          442      442
24          300      300
25          891      891
26          462      462
27          770      770
28          912      912
29          959      959
...         ...      ...
246833      920      920
246834      379      379
246835      283      283
246836      306      306
246837      815      815
246838       40       40
246839      832      832
246840      330      330


ValueError: Unknown label type: 'unknown'

In [ ]:
for each in range(0, len(x)):
    print len(x[each])
#         print x[each][i]

In [16]:
print new_val['closureDate']

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
246833    0
246834    0
246835    0
246836    0
246837    0
246838    0
246839    0
246840    0
246841    0
246842    0
246843    0
246844    0
246845    0
246846    0
246847    0
246848    0
246849    1
246850    0
246851    0
246852    0
246853    0
246854    0
246855    0
246856    0
246857    0
246858    0
246859    0
246860    0
246861    0
246862    0
Name: closureDate, Length: 246863, dtype: object


In [ ]:
print new_data